In [27]:
import cv2
import numpy as np
import math

# Open the webcam
cap0 = cv2.VideoCapture(0) # https://www.logitech.com/en-us/shop/p/c922-pro-stream-webcam.960-001087

# Read one frame to get image size
ret, frame = cap0.read()
cap0.release()

if not ret:
    raise RuntimeError("Could not read from the webcam.")

# Image size
height, width = frame.shape[:2]
print(f"Image resolution: {width} x {height}")

# Estimated horizontal FoV in degrees (you can adjust this if needed)
fov_x_deg = 78
fov_x_rad = math.radians(fov_x_deg)

# Compute focal length in pixels
f_x = width / (2 * math.tan(fov_x_rad / 2))
f_y = f_x  # assume square pixels

# Assume principal point is at the center
c_x = width / 2
c_y = height / 2

# Intrinsic camera matrix
K = np.array([
    [f_x,   0,    c_x],
    [0,     f_y,  c_y],
    [0,     0,    1]
])

print("Approximate Intrinsic Camera Matrix (K):")
print(K)

K0 = K


Image resolution: 640 x 480
Approximate Intrinsic Camera Matrix (K):
[[     395.17           0         320]
 [          0      395.17         240]
 [          0           0           1]]


In [30]:
import cv2
import numpy as np
import math

# Open the webcam
cap1 = cv2.VideoCapture(1) #laptop cam

# Read one frame to get image size
ret, frame = cap1.read()
cap1.release()

if not ret:
    raise RuntimeError("Could not read from the webcam.")

# Image size
height, width = frame.shape[:2]
print(f"Image resolution: {width} x {height}")

# Estimated horizontal FoV in degrees (you can adjust this if needed)
fov_x_deg = 78.3
fov_x_rad = math.radians(fov_x_deg)

# Compute focal length in pixels
f_x = width / (2 * math.tan(fov_x_rad / 2))
f_y = f_x  # assume square pixels

# Assume principal point is at the center
c_x = width / 2
c_y = height / 2

# Intrinsic camera matrix
K = np.array([
    [f_x,   0,    c_x],
    [0,     f_y,  c_y],
    [0,     0,    1]
])

print("Approximate Intrinsic Camera Matrix (K):")
print(K)

K1 = K


Image resolution: 640 x 480
Approximate Intrinsic Camera Matrix (K):
[[     393.06           0         320]
 [          0      393.06         240]
 [          0           0           1]]


In [6]:
import cv2
import numpy as np
from scipy.spatial.transform import Rotation as Rscipy

# Open both cameras
cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)

ret0, img0 = cap0.read()
ret1, img1 = cap1.read()

cap0.release()
cap1.release()

if not ret0 or not ret1:
    raise RuntimeError("Could not read from one or both cameras.")

# Convert to grayscale
gray0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

# Detect and match features
orb = cv2.ORB_create(1000)
kp0, des0 = orb.detectAndCompute(gray0, None)
kp1, des1 = orb.detectAndCompute(gray1, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des0, des1)
matches = sorted(matches, key=lambda x: x.distance)

good_matches = matches[:100]
pts0 = np.float32([kp0[m.queryIdx].pt for m in good_matches])
pts1 = np.float32([kp1[m.trainIdx].pt for m in good_matches])

# Intrinsics
K = np.array([
    [395.16709009, 0, 320],
    [0, 395.16709009, 240],
    [0, 0, 1]
])

# Estimate essential matrix and recover pose
E, mask = cv2.findEssentialMat(pts0, pts1, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
_, R, t, mask_pose = cv2.recoverPose(E, pts0, pts1, K)

# ---- Part 1: Draw matches ----
matched_img = cv2.drawMatches(img0, kp0, img1, kp1, good_matches, None, flags=2)
cv2.imshow("Feature Matches", matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ---- Part 2: Convert Rotation Matrix to Euler Angles ----
r = Rscipy.from_matrix(R)
euler_angles = r.as_euler('xyz', degrees=True)

print("\nRotation Matrix (R):")
print(R)

print("\nTranslation Vector (t):")
print(t)

print("\nEuler Angles (XYZ, degrees):")
print(euler_angles)



Rotation Matrix (R):
[[    0.99162   -0.052836     0.11787]
 [   0.020276     0.96487     0.26194]
 [   -0.12756    -0.25735     0.95786]]

Translation Vector (t):
[[    0.97941]
 [   -0.12776]
 [    0.15628]]

Euler Angles (XYZ, degrees):
[    -15.039      7.3289      1.1714]


In [4]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load the YOLOv11 model for pose estimation
model = YOLO("yolo11n-pose.pt")

# Open two video captures (e.g., two webcams)
cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)

if not cap0.isOpened() or not cap1.isOpened():
    print("Error: Cannot open one or both webcams.")
    exit()

while True:
    # Read frames from both cameras
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    
    if not ret0 or not ret1:
        print("Error: Can't read frames from one or both cameras.")
        break

    # Run pose estimation on both frames
    results0 = model(frame0)
    results1 = model(frame1)

    # Get keypoints from both results (assuming single person detection for simplicity)
    kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
    kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

    # Plot the results (draw skeleton/keypoints)
    annotated_frame0 = results0[0].plot()
    annotated_frame1 = results1[0].plot()

    # Combine frames side by side for display
    combined_frame = np.hstack((annotated_frame0, annotated_frame1))

    # If we have keypoints from both cameras, process them
    if kpts0 is not None and kpts1 is not None:
        # Filter out [0,0] points and create matched pairs
        valid_pairs = []
        for i, (kp0, kp1) in enumerate(zip(kpts0, kpts1)):
            # Skip if either point is [0,0]
            if np.all(kp0 == [0,0]) or np.all(kp1 == [0,0]):
                continue
            valid_pairs.append((i, kp0, kp1))

        # Draw lines between matched keypoints (for visualization)
        for pair in valid_pairs:
            idx, kp0, kp1 = pair
            # Adjust coordinates for the combined frame (kp1 is in the right half)
            pt1 = tuple(map(int, kp0))
            pt2 = (frame0.shape[1] + int(kp1[0]), int(kp1[1]))
            
            # Draw line between matched points
            cv2.line(combined_frame, pt1, pt2, (0, 255, 0), 2)
            
            # Label with the keypoint index
            cv2.putText(combined_frame, str(idx), pt1, cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(combined_frame, str(idx), pt2, cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)

    # Display the combined frame
    cv2.imshow("Dual Camera Pose Estimation with Matched Keypoints", combined_frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap0.release()
cap1.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 44.0ms
Speed: 3.2ms preprocess, 44.0ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 42.8ms
Speed: 4.0ms preprocess, 42.8ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 39.1ms
Speed: 4.4ms preprocess, 39.1ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 39.0ms
Speed: 3.8ms preprocess, 39.0ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 38.9ms
Speed: 3.6ms preprocess, 38.9ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 30.0ms
Speed: 3.5ms preprocess, 30.0ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 19.5ms
Speed: 2.2ms preprocess, 19.5ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 19.8ms
Speed: 2.8ms preprocess, 19.8ms inference, 2.1ms postprocess per image at shape (1, 3, 48

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.spatial.transform import Rotation as Rscipy
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# ---- STEP 1: Estimate Extrinsics (R, t) once ----
def estimate_camera_pose(cap0, cap1, K):
    ret0, img0 = cap0.read()
    ret1, img1 = cap1.read()
    if not ret0 or not ret1:
        raise RuntimeError("Failed to capture calibration images")

    gray0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create(1000)
    kp0, des0 = orb.detectAndCompute(gray0, None)
    kp1, des1 = orb.detectAndCompute(gray1, None)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(des0, des1)
    matches = sorted(matches, key=lambda x: x.distance)
    matches = matches[:100]

    pts0 = np.float32([kp0[m.queryIdx].pt for m in matches])
    pts1 = np.float32([kp1[m.trainIdx].pt for m in matches])

    E, _ = cv2.findEssentialMat(pts0, pts1, K, method=cv2.RANSAC, threshold=1.0)
    _, R, t, _ = cv2.recoverPose(E, pts0, pts1, K)
    return R, t

# ---- STEP 2: Main loop ----
K0 = np.array([[395.16709009, 0, 320],
               [0, 395.16709009, 240],
               [0, 0, 1]])
K1 = np.array([[393.05858722, 0, 320],
               [0, 393.05858722, 240],
               [0, 0, 1]])

# Projection matrices
P0 = K0 @ np.hstack((np.eye(3), np.zeros((3, 1))))  # [I|0]

cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)
if not cap0.isOpened() or not cap1.isOpened():
    raise RuntimeError("Cameras could not be opened.")

# Estimate extrinsics once
R, t = estimate_camera_pose(cap0, cap1, K0)
P1 = K1 @ np.hstack((R, t))  # [R|t]

print(R,t)

model = YOLO("yolo11n-pose.pt")

while True:
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    if not ret0 or not ret1:
        break

    results0 = model(frame0)
    results1 = model(frame1)

    kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
    kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

    annotated_frame0 = results0[0].plot()
    annotated_frame1 = results1[0].plot()
    combined_frame = np.hstack((annotated_frame0, annotated_frame1))

    if kpts0 is not None and kpts1 is not None:
        valid_pairs = []
        for i, (kp0, kp1) in enumerate(zip(kpts0, kpts1)):
            if np.all(kp0 == [0, 0]) or np.all(kp1 == [0, 0]):
                continue
            valid_pairs.append((i, kp0, kp1))

        for pair in valid_pairs:
            idx, kp0, kp1 = pair
            
            # Triangulate 3D point
            kp0_h = np.array([[kp0[0]], [kp0[1]]])
            kp1_h = np.array([[kp1[0]], [kp1[1]]])
            point_4d_h = cv2.triangulatePoints(P0, P1, kp0_h, kp1_h)
            point_3d = point_4d_h[:3] / point_4d_h[3]
            distance = np.linalg.norm(point_3d)

            # Draw connecting lines
            pt1 = tuple(map(int, kp0))
            pt2 = (frame0.shape[1] + int(kp1[0]), int(kp1[1]))
            cv2.line(combined_frame, pt1, pt2, (0, 255, 0), 2)
            cv2.putText(combined_frame, str(idx), pt1, cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(combined_frame, str(idx), pt2, cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)

            # Draw circles and bigger distance text ONLY for left and right hands (9 and 10)
            if idx == 9 or idx == 10:
                # Scale distance between 1m and 3m:
                # clamp distance between 1 and 3 meters
                dist_clamped = np.clip(distance, 1.0, 3.0)
                
                # Map 1m->max_radius, 3m->min_radius
                max_radius = 40
                min_radius = 10
                radius = int(np.interp(dist_clamped, [1.0, 3.0], [max_radius, min_radius]))
                
                # Choose color and position
                if idx == 9:  # left hand on left frame
                    center = pt1
                    color = (0, 0, 255)  # Red
                else:  # right hand on right frame
                    center = pt2
                    color = (255, 0, 0)  # Blue
                
                cv2.circle(combined_frame, center, radius, color, thickness=2)

                # Bigger font for distance text
                cv2.putText(combined_frame, f"{distance:.2f}m", (center[0], center[1] - radius - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2, cv2.LINE_AA)



    cv2.imshow("Triangulated Pose with Distance", combined_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap0.release()
cap1.release()
cv2.destroyAllWindows()


[[     0.9992   -0.027842   -0.028698]
 [   0.034448     0.96382     0.26432]
 [     0.0203    -0.26509     0.96401]] [[    0.97825]
 [  -0.074395]
 [    0.19363]]

0: 480x640 1 person, 21.6ms
Speed: 3.4ms preprocess, 21.6ms inference, 7.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.6ms
Speed: 1.7ms preprocess, 13.6ms inference, 8.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 12.2ms
Speed: 2.7ms preprocess, 12.2ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.9ms
Speed: 1.8ms preprocess, 9.9ms inference, 5.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.3ms
Speed: 2.7ms preprocess, 11.3ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.4ms
Speed: 2.8ms preprocess, 11.4ms inference, 6.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 10.5ms
Speed: 1.9ms preprocess, 10.5ms inference, 3.0ms post

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import socket
import json
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# ---- STEP 1: Estimate Extrinsics (R, t) once ----
def estimate_camera_pose(cap0, cap1, K):
    ret0, img0 = cap0.read()
    ret1, img1 = cap1.read()
    if not ret0 or not ret1:
        raise RuntimeError("Failed to capture calibration images")

    # Flip the images horizontally to correct mirroring
    img0 = cv2.flip(img0, 1)
    img1 = cv2.flip(img1, 1)

    gray0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)


    orb = cv2.ORB_create(1000)
    kp0, des0 = orb.detectAndCompute(gray0, None)
    kp1, des1 = orb.detectAndCompute(gray1, None)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(des0, des1)
    matches = sorted(matches, key=lambda x: x.distance)
    matches = matches[:100]

    pts0 = np.float32([kp0[m.queryIdx].pt for m in matches])
    pts1 = np.float32([kp1[m.trainIdx].pt for m in matches])

    E, _ = cv2.findEssentialMat(pts0, pts1, K, method=cv2.RANSAC, threshold=1.0)
    _, R, t, _ = cv2.recoverPose(E, pts0, pts1, K)
    return R, t

def normalize_point(x, y, width, height):
    norm_x = (x / width) * 2 - 1
    norm_y = (y / height) * 2 - 1
    return [norm_x, -norm_y]



def normalize_depth(distance, z_min=2.0, z_max=3.0):
    # Linear map: z_min -> 0, z_max -> 1, allow outside range
    return (distance - z_min) / (z_max - z_min)

# Camera intrinsics
K0 = np.array([[395.16709009, 0, 320],
               [0, 395.16709009, 240],
               [0, 0, 1]])
K1 = np.array([[393.05858722, 0, 320],
               [0, 393.05858722, 240],
               [0, 0, 1]])

P0 = K0 @ np.hstack((np.eye(3), np.zeros((3, 1))))  # Camera 0 projection matrix [I|0]

cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)
if not cap0.isOpened() or not cap1.isOpened():
    raise RuntimeError("Cameras could not be opened.")

print("Estimating extrinsics...")
R, t = estimate_camera_pose(cap0, cap1, K0)
P1 = K1 @ np.hstack((R, t))  # Camera 1 projection matrix [R|t]
print("Rotation:\n", R)
print("Translation:\n", t.T)

# Setup socket for Godot connection
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(('localhost', 5050))
sock.listen(1)
print("Waiting for Godot to connect...")
conn, addr = sock.accept()
print(f"Connected by {addr}")

model = YOLO("yolo11n-pose.pt")

while True:
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    if not ret0 or not ret1:
        print("Failed to grab frames")
        break

    # Flip frames to correct mirroring
    frame0 = cv2.flip(frame0, 1)
    frame1 = cv2.flip(frame1, 1)

    height0, width0 = frame0.shape[:2]
    height1, width1 = frame1.shape[:2]

    results0 = model(frame0)
    results1 = model(frame1)

    # Extract keypoints: (N, 2) numpy arrays
    kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
    kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

    # Default output for hands if missing
    left_hand_out = [0, 0, 0]
    right_hand_out = [0, 0, 0]

    if kpts0 is not None and kpts1 is not None and len(kpts0) >= 11 and len(kpts1) >= 11:
        # Get left and right hand keypoints
        lh0 = kpts0[9]
        rh0 = kpts0[10]
        lh1 = kpts1[9]
        rh1 = kpts1[10]

        # Check for valid detections (non-zero)
        def valid_kp(kp): return not np.allclose(kp, [0,0])

        # Triangulate left hand if valid in both cameras
        if valid_kp(lh0) and valid_kp(lh1):
            pts0 = np.array([[lh0[0]], [lh0[1]]], dtype=np.float64)
            pts1 = np.array([[lh1[0]], [lh1[1]]], dtype=np.float64)
            point_4d = cv2.triangulatePoints(P0, P1, pts0, pts1)
            point_3d = (point_4d[:3] / point_4d[3]).flatten()
            dist = np.linalg.norm(point_3d)
            z_norm = normalize_depth(dist)
            x_norm, y_norm = normalize_point(lh0[0], lh0[1], width0, height0)
            left_hand_out = [x_norm, y_norm, z_norm]

        # Triangulate right hand if valid in both cameras
        if valid_kp(rh0) and valid_kp(rh1):
            pts0 = np.array([[rh0[0]], [rh0[1]]], dtype=np.float64)
            pts1 = np.array([[rh1[0]], [rh1[1]]], dtype=np.float64)
            point_4d = cv2.triangulatePoints(P0, P1, pts0, pts1)
            point_3d = (point_4d[:3] / point_4d[3]).flatten()
            dist = np.linalg.norm(point_3d)
            z_norm = normalize_depth(dist)
            x_norm, y_norm = normalize_point(rh0[0], rh0[1], width0, height0)
            right_hand_out = [x_norm, y_norm, z_norm]

    data = json.dumps({'left': left_hand_out, 'right': right_hand_out})
    try:
        conn.sendall(data.encode('utf-8') + b'\n')
    except BrokenPipeError:
        print("Godot disconnected.")
        break

    # Optional: Show combined annotated frames side by side with lines for hands
    annotated0 = results0[0].plot()
    annotated1 = results1[0].plot()
    combined = np.hstack((annotated0, annotated1))

    # Draw lines between corresponding hands (optional visualization)
    for idx, (pt0, pt1) in [(9, (lh0, lh1)), (10, (rh0, rh1))]:
        if valid_kp(pt0) and valid_kp(pt1):
            pt0_int = tuple(map(int, pt0))
            pt1_int = (annotated0.shape[1] + int(pt1[0]), int(pt1[1]))
            cv2.line(combined, pt0_int, pt1_int, (0, 255, 0), 2)
            # Draw circles with radius based on distance if you want

    cv2.imshow("Triangulated Pose with Distance", combined)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

conn.close()
cap0.release()
cap1.release()
cv2.destroyAllWindows()


Estimating extrinsics...
Rotation:
 [[    0.96802   -0.061707     0.24315]
 [  0.0023623     0.97147     0.23713]
 [   -0.25085    -0.22898     0.94056]]
Translation:
 [[   -0.64608     0.11573     0.75445]]
Waiting for Godot to connect...
Connected by ('127.0.0.1', 51488)

0: 480x640 1 person, 10.9ms
Speed: 2.1ms preprocess, 10.9ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.3ms
Speed: 1.5ms preprocess, 9.3ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 14.6ms
Speed: 1.6ms preprocess, 14.6ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.2ms
Speed: 1.3ms preprocess, 11.2ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.1ms
Speed: 1.3ms preprocess, 8.1ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.8ms
Speed: 1.0ms preprocess, 7.8ms inference, 1.7ms postprocess per image at

In [33]:
import time
import cv2
import numpy as np

def show_instructions(frame, text):
    """Overlay text instructions on the frame"""
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, text, (30, 50), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Calibration will capture in:", (30, 90), font, 0.7, (0, 255, 255), 1, cv2.LINE_AA)

def get_hand_distance_with_timer(cap0, cap1, model, P0, P1, prompt, countdown=10):
    start_time = time.time()
    captured_distance = None
    while True:
        ret0, frame0 = cap0.read()
        ret1, frame1 = cap1.read()
        if not ret0 or not ret1:
            print("ERORR")
            continue

        # Optionally flip frames horizontally if your cameras are mirrored
        frame0 = cv2.flip(frame0, 1)
        frame1 = cv2.flip(frame1, 1)

        # Calculate remaining countdown time
        elapsed = time.time() - start_time
        time_left = int(countdown - elapsed)

        results0 = model(frame0)
        results1 = model(frame1)

        combined = np.hstack((results0[0].plot(), results1[0].plot()))
        show_instructions(combined, prompt)

        # Show countdown timer on the frame
        if time_left > 0:
            cv2.putText(combined, f"{time_left} seconds", (30, 130), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        else:
            # Capture once timer finishes
            kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
            kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

            if kpts0 is not None and kpts1 is not None and len(kpts0) >= 11 and len(kpts1) >= 11:
                lh0 = kpts0[9]
                lh1 = kpts1[9]
                if not np.allclose(lh0, [0, 0]) and not np.allclose(lh1, [0, 0]):
                    pts0 = np.array([[lh0[0]], [lh0[1]]], dtype=np.float64)
                    pts1 = np.array([[lh1[0]], [lh1[1]]], dtype=np.float64)
                    point_4d = cv2.triangulatePoints(P0, P1, pts0, pts1)
                    point_3d = (point_4d[:3] / point_4d[3]).flatten()
                    captured_distance = np.linalg.norm(point_3d)
                    cv2.putText(combined, f"Captured distance: {captured_distance:.2f}", (30, 180),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                    cv2.imshow("Calibration", combined)
                    cv2.waitKey(2000)  # Show for 2 seconds
                    break
                else:
                    cv2.putText(combined, "Hands not detected properly", (30, 180),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            else:
                cv2.putText(combined, "Hands not detected properly", (30, 180),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        cv2.imshow("Calibration", combined)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC to abort calibration
            print("Calibration aborted.")
            captured_distance = None
            break

    return captured_distance

def calibrate_depth_range_with_timer(cap0, cap1, model, P0, P1):
    near = get_hand_distance_with_timer(cap0, cap1, model, P0, P1, "Step 1: Put your hands close to the cameras", countdown=5)
    if near is None:
        return None, None
    far = get_hand_distance_with_timer(cap0, cap1, model, P0, P1, "Step 2: Put your hands far away from the cameras", countdown=5)
    if far is None:
        return None, None

    cv2.destroyWindow("Calibration")
    print(f"Calibration completed. z_min={near:.2f}, z_max={far:.2f}")
    return min(near, far), max(near, far)


In [38]:
import cv2
import numpy as np
from ultralytics import YOLO
import socket
import json
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# ---- STEP 1: Estimate Extrinsics (R, t) once ----
def estimate_camera_pose(cap0, cap1, K):
    ret0, img0 = cap0.read()
    ret1, img1 = cap1.read()
    if not ret0 or not ret1:
        raise RuntimeError("Failed to capture calibration images")

    # Flip the images horizontally to correct mirroring
    img0 = cv2.flip(img0, 1)
    img1 = cv2.flip(img1, 1)

    gray0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)


    orb = cv2.ORB_create(1000)
    kp0, des0 = orb.detectAndCompute(gray0, None)
    kp1, des1 = orb.detectAndCompute(gray1, None)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(des0, des1)
    matches = sorted(matches, key=lambda x: x.distance)
    matches = matches[:100]

    pts0 = np.float32([kp0[m.queryIdx].pt for m in matches])
    pts1 = np.float32([kp1[m.trainIdx].pt for m in matches])

    E, _ = cv2.findEssentialMat(pts0, pts1, K, method=cv2.RANSAC, threshold=1.0)
    _, R, t, _ = cv2.recoverPose(E, pts0, pts1, K)
    return R, t

def normalize_point(x, y, width, height):
    norm_x = (x / width) * 2 - 1
    norm_y = (y / height) * 2 - 1
    return [norm_x, -norm_y]



def normalize_depth(distance, z_min=2.0, z_max=3.0):
    # Linear map: z_min -> 0, z_max -> 1, allow outside range
    return (distance - z_min) / (z_max - z_min)

# Camera intrinsics
K0 = np.array([[395.16709009, 0, 320],
               [0, 395.16709009, 240],
               [0, 0, 1]])
K1 = np.array([[393.05858722, 0, 320],
               [0, 393.05858722, 240],
               [0, 0, 1]])

P0 = K0 @ np.hstack((np.eye(3), np.zeros((3, 1))))  # Camera 0 projection matrix [I|0]

cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)
if not cap0.isOpened() or not cap1.isOpened():
    raise RuntimeError("Cameras could not be opened.")

print("Estimating extrinsics...")
R, t = estimate_camera_pose(cap0, cap1, K0)
P1 = K1 @ np.hstack((R, t))  # Camera 1 projection matrix [R|t]
print("Rotation:\n", R)
print("Translation:\n", t.T)


z_min, z_max = calibrate_depth_range_with_timer(cap0, cap1, model, P0, P1)
if z_min is None or z_max is None:
    raise RuntimeError("Depth calibration failed or aborted.")

def normalize_depth(distance):
    return (distance - z_min) / (z_max - z_min)


Estimating extrinsics...
Rotation:
 [[    0.98273   0.0075829     0.18491]
 [  -0.060834     0.95687     0.28407]
 [   -0.17478    -0.29041      0.9408]]
Translation:
 [[   -0.95329   -0.034095     0.30013]]

0: 480x640 1 person, 7.3ms
Speed: 2.1ms preprocess, 7.3ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.7ms
Speed: 1.4ms preprocess, 7.7ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.6ms
Speed: 1.8ms preprocess, 8.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.8ms
Speed: 1.2ms preprocess, 8.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.8ms
Speed: 1.6ms preprocess, 8.8ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.3ms
Speed: 1.4ms preprocess, 8.3ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.6ms
Speed: 1.5ms prepro

In [ ]:
def normalize_point(x, y, width, height):
    norm_x = (x / width) * 2 - 1
    norm_y = (y / height) * 2 - 1
    return [norm_x, -norm_y]

# Setup socket for Godot connection
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(('localhost', 5050))
sock.listen(1)
print("Waiting for Godot to connect...")
conn, addr = sock.accept()
print(f"Connected by {addr}")

model = YOLO("yolo11n-pose.pt")

while True:
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    if not ret0 or not ret1:
        print("Failed to grab frames")
        break

    # Flip frames to correct mirroring
    frame0 = cv2.flip(frame0, 1)
    frame1 = cv2.flip(frame1, 1)

    height0, width0 = frame0.shape[:2]
    height1, width1 = frame1.shape[:2]

    results0 = model(frame0)
    results1 = model(frame1)

    # Extract keypoints: (N, 2) numpy arrays
    kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
    kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

    # Default output for hands if missing
    left_hand_out = [0, 0, 0]
    right_hand_out = [0, 0, 0]

    if kpts0 is not None and kpts1 is not None and len(kpts0) >= 11 and len(kpts1) >= 11:
        # Get left and right hand keypoints
        lh0 = kpts0[9]
        rh0 = kpts0[10]
        lh1 = kpts1[9]
        rh1 = kpts1[10]

        # Check for valid detections (non-zero)
        def valid_kp(kp): return not np.allclose(kp, [0,0])

        # Triangulate left hand if valid in both cameras
        if valid_kp(lh0) and valid_kp(lh1):
            pts0 = np.array([[lh0[0]], [lh0[1]]], dtype=np.float64)
            pts1 = np.array([[lh1[0]], [lh1[1]]], dtype=np.float64)
            point_4d = cv2.triangulatePoints(P0, P1, pts0, pts1)
            point_3d = (point_4d[:3] / point_4d[3]).flatten()
            dist = np.linalg.norm(point_3d)
            z_norm = normalize_depth(dist)
            x_norm, y_norm = normalize_point(lh0[0], lh0[1], width0, height0)
            left_hand_out = [x_norm, y_norm, z_norm]

        # Triangulate right hand if valid in both cameras
        if valid_kp(rh0) and valid_kp(rh1):
            pts0 = np.array([[rh0[0]], [rh0[1]]], dtype=np.float64)
            pts1 = np.array([[rh1[0]], [rh1[1]]], dtype=np.float64)
            point_4d = cv2.triangulatePoints(P0, P1, pts0, pts1)
            point_3d = (point_4d[:3] / point_4d[3]).flatten()
            dist = np.linalg.norm(point_3d)
            z_norm = normalize_depth(dist)
            x_norm, y_norm = normalize_point(rh0[0], rh0[1], width0, height0)
            right_hand_out = [x_norm, y_norm, z_norm]

    data = json.dumps({'left': left_hand_out, 'right': right_hand_out})
    try:
        conn.sendall(data.encode('utf-8') + b'\n')
    except BrokenPipeError:
        print("Godot disconnected.")
        break

    # Optional: Show combined annotated frames side by side with lines for hands
    annotated0 = results0[0].plot()
    annotated1 = results1[0].plot()
    combined = np.hstack((annotated0, annotated1))

    # Draw lines between corresponding hands (optional visualization)
    for idx, (pt0, pt1) in [(9, (lh0, lh1)), (10, (rh0, rh1))]:
        if valid_kp(pt0) and valid_kp(pt1):
            pt0_int = tuple(map(int, pt0))
            pt1_int = (annotated0.shape[1] + int(pt1[0]), int(pt1[1]))
            cv2.line(combined, pt0_int, pt1_int, (0, 255, 0), 2)
            # Draw circles with radius based on distance if you want

    cv2.imshow("Triangulated Pose with Distance", combined)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

conn.close()
cap0.release()
cap1.release()
cv2.destroyAllWindows()


Waiting for Godot to connect...
Connected by ('127.0.0.1', 51948)

0: 480x640 1 person, 12.6ms
Speed: 1.6ms preprocess, 12.6ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.8ms
Speed: 1.4ms preprocess, 11.8ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.3ms
Speed: 3.2ms preprocess, 9.3ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 10.3ms
Speed: 2.6ms preprocess, 10.3ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.1ms
Speed: 1.3ms preprocess, 8.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.0ms
Speed: 1.1ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.4ms
Speed: 1.5ms preprocess, 7.4ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.5ms
Speed: 1.1ms preprocess, 7.5ms i